In [1]:
import numpy as np
import pandas as pd
#import wfdb
import os
from glob import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
import scipy.signal as signal
from scipy.signal import butter, lfilter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.alexnet import alexnet
seed=2020
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
reclist=glob("../apnea/seg/*")#"./seg/*"
dtlist=glob("../apnea/seg/*/*.npy*")#"./seg/*/*.npy*"
reclist.sort()
dtlist.sort()
lab_list=[seg.split('_')[0][-1] for seg in dtlist]#shuffle by idx
print(lab_list.count('A')/len(lab_list),lab_list.count('N')/len(lab_list))

#reclist=[rec for rec in reclist if rec[-3]!='b'] #Remove Borderline
# reclist[0:5]

0.3807303354413779 0.6192696645586221


In [5]:
def cheb_bandpass_filter(data, lowcut, highcut, signal_freq, filter_order):
        """
        Method responsible for creating and applying Butterworth filter.
        :param deque data: raw data
        :param float lowcut: filter lowcut frequency value
        :param float highcut: filter highcut frequency value
        :param int signal_freq: signal frequency in samples per second (Hz)
        :param int filter_order: filter order
        :return array: filtered data
        """
        nyquist_freq = 0.5 * signal_freq
        low = lowcut / nyquist_freq
        high = highcut / nyquist_freq
        #b, a = butter(filter_order, [low, high], btype="band")
        b, a = signal.cheby2(filter_order, 40, [low, high], 'band', analog=False)
        y = lfilter(b, a, data)
        return y

In [6]:
subjects=[['a11'],['a15'],['a17'],['b01'],['c07'],
          ['a11','a15','a17','b01','c07'],
          ['a14','a19','b05','c01','c07'],
          ['a04','a19','b05','c01','c09'],['b02','b03'],['x16','x21']]
len(subjects)

10

In [7]:
from sklearn.model_selection import KFold
def getTrainTestList(reclist,opt='rec_cv',fold=1):
    train_dtlist=[]
    test_dtlist=[]
    if opt=='rec_cv':#reclist is rec list
        kf = KFold(n_splits=10)
        kf_idx=kf.split(reclist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_rec,test_rec=list(np.array(reclist)[train_idx]),list(np.array(reclist)[test_idx])
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')
    elif opt=='physionet':
        train_rec=[rec for rec in reclist if rec[-3]!='x']
        test_rec=[rec for rec in reclist if rec[-3]=='x']
        print(len(train_rec),len(test_rec))
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')
    elif opt=='physionet_train':
        train_rec=[rec for rec in reclist if rec[-3]!='x']
        #print(len(train_rec),len(test_rec))
        kf_idx=kf.split(dtlist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_dtlist,test_dtlist=list(np.array(dtlist)[train_idx]),list(np.array(dtlist)[test_idx])
    elif opt=='blind':
        subjects=[['a11'],
                  ['a15','x27','x28'],
                  ['a17','x12'],
                  ['b01','x03'],
                  ['c07','x34'],
                  ['a11','a15','x27','x28','a17','x12','b01','x03','c07','x34'],
                  ['a14','a19','x05','x08','x25','b05','x11','c01','x35','c07','x34'],
                  ['a04','a19','x05','x08','x25','b05','x11','c01','x35','c09'],
                  ['b02','b03','x16','x21']]
        train_rec=[rec for rec in reclist if rec.split('/')[-1] not in subjects[fold-1]]
        test_rec=[rec for rec in reclist if rec.split('/')[-1] in subjects[fold-1]]
        print(len(train_rec),len(test_rec),test_rec)
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')

    else:#reclist is npy data list
        kf = KFold(n_splits=10)
        kf_idx=kf.split(dtlist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_dtlist,test_dtlist=list(np.array(dtlist)[train_idx]),list(np.array(dtlist)[test_idx])
         
    return train_dtlist,test_dtlist

def dtclean(dt_path):
    dt=np.load(dt_path)
    if dt.std()<0.1:
        print(dt_path)
        return 1      
    else:
        return 0
#train_rec,test_rec=reclist[train_idx],reclist[test_idx]
train_dtlist,test_dtlist=getTrainTestList(reclist,opt='kfold',fold=6)#fold 1-10
print('*********')
print(len(train_dtlist),len(test_dtlist))
train_dtlist=[file for file in train_dtlist if dtclean(file)==0]
np.random.shuffle(train_dtlist)
test_dtlist=[file for file in test_dtlist if dtclean(file)==0]
print(len(train_dtlist),len(test_dtlist))

*********
30882 3431
../apnea/seg\a12\A_a12_112.npy
../apnea/seg\a12\A_a12_143.npy
../apnea/seg\a12\A_a12_144.npy
../apnea/seg\a12\A_a12_145.npy
../apnea/seg\a12\A_a12_163.npy
../apnea/seg\a12\A_a12_176.npy
../apnea/seg\a12\A_a12_177.npy
../apnea/seg\a12\A_a12_178.npy
../apnea/seg\a12\A_a12_179.npy
../apnea/seg\a12\A_a12_180.npy
../apnea/seg\a12\A_a12_181.npy
../apnea/seg\a12\A_a12_182.npy
../apnea/seg\a12\A_a12_183.npy
../apnea/seg\a12\A_a12_187.npy
../apnea/seg\a12\A_a12_191.npy
../apnea/seg\a12\A_a12_192.npy
../apnea/seg\a12\A_a12_193.npy
../apnea/seg\a12\A_a12_194.npy
../apnea/seg\a12\A_a12_195.npy
../apnea/seg\a12\A_a12_197.npy
../apnea/seg\a12\A_a12_206.npy
../apnea/seg\a12\A_a12_207.npy
../apnea/seg\a12\A_a12_208.npy
../apnea/seg\a12\A_a12_209.npy
../apnea/seg\a12\A_a12_210.npy
../apnea/seg\a12\A_a12_211.npy
../apnea/seg\a12\A_a12_212.npy
../apnea/seg\a12\A_a12_228.npy
../apnea/seg\a12\A_a12_229.npy
../apnea/seg\a12\A_a12_230.npy
../apnea/seg\a12\A_a12_231.npy
../apnea/seg\a12\A

../apnea/seg\a12\A_a12_538.npy
../apnea/seg\a12\A_a12_539.npy
../apnea/seg\a12\A_a12_540.npy
../apnea/seg\a12\A_a12_541.npy
../apnea/seg\a12\A_a12_542.npy
../apnea/seg\a12\A_a12_543.npy
../apnea/seg\a12\A_a12_545.npy
../apnea/seg\a12\A_a12_546.npy
../apnea/seg\a12\A_a12_547.npy
../apnea/seg\a12\A_a12_548.npy
../apnea/seg\a12\A_a12_549.npy
../apnea/seg\a12\A_a12_550.npy
../apnea/seg\a12\A_a12_551.npy
../apnea/seg\a12\A_a12_552.npy
../apnea/seg\a12\A_a12_554.npy
../apnea/seg\a12\A_a12_555.npy
../apnea/seg\a12\A_a12_556.npy
../apnea/seg\a12\A_a12_557.npy
../apnea/seg\a12\A_a12_558.npy
../apnea/seg\a12\A_a12_559.npy
../apnea/seg\a12\A_a12_560.npy
../apnea/seg\a12\A_a12_561.npy
../apnea/seg\a12\A_a12_562.npy
../apnea/seg\a12\A_a12_563.npy
../apnea/seg\a12\A_a12_564.npy
../apnea/seg\a12\A_a12_565.npy
../apnea/seg\a12\A_a12_566.npy
../apnea/seg\a12\A_a12_567.npy
../apnea/seg\a12\A_a12_568.npy
../apnea/seg\a12\A_a12_569.npy
../apnea/seg\a12\A_a12_570.npy
../apnea/seg\a12\A_a12_571.npy
../apnea

../apnea/seg\c08\N_c08_330.npy
../apnea/seg\c08\N_c08_332.npy
../apnea/seg\c08\N_c08_338.npy
../apnea/seg\c08\N_c08_339.npy
../apnea/seg\c08\N_c08_340.npy
../apnea/seg\c08\N_c08_341.npy
../apnea/seg\c08\N_c08_35.npy
../apnea/seg\c08\N_c08_355.npy
../apnea/seg\c08\N_c08_356.npy
../apnea/seg\c08\N_c08_357.npy
../apnea/seg\c08\N_c08_363.npy
../apnea/seg\c08\N_c08_382.npy
../apnea/seg\c08\N_c08_409.npy
../apnea/seg\c08\N_c08_410.npy
../apnea/seg\c08\N_c08_411.npy
../apnea/seg\c08\N_c08_412.npy
../apnea/seg\c08\N_c08_413.npy
../apnea/seg\c08\N_c08_414.npy
../apnea/seg\c08\N_c08_440.npy
../apnea/seg\c08\N_c08_441.npy
../apnea/seg\c08\N_c08_442.npy
../apnea/seg\c08\N_c08_443.npy
../apnea/seg\c08\N_c08_444.npy
../apnea/seg\c08\N_c08_445.npy
../apnea/seg\c08\N_c08_446.npy
../apnea/seg\c08\N_c08_447.npy
../apnea/seg\c08\N_c08_448.npy
../apnea/seg\c08\N_c08_449.npy
../apnea/seg\c08\N_c08_450.npy
../apnea/seg\c08\N_c08_451.npy
../apnea/seg\c08\N_c08_452.npy
../apnea/seg\c08\N_c08_453.npy
../apnea/

../apnea/seg\x17\N_x17_4_243.npy
../apnea/seg\x17\N_x17_4_244.npy
../apnea/seg\x17\N_x17_4_245.npy
../apnea/seg\x17\N_x17_4_246.npy
../apnea/seg\x17\N_x17_4_248.npy
../apnea/seg\x17\N_x17_4_249.npy
../apnea/seg\x17\N_x17_4_250.npy
../apnea/seg\x17\N_x17_4_251.npy
../apnea/seg\x17\N_x17_4_252.npy
../apnea/seg\x17\N_x17_4_253.npy
../apnea/seg\x17\N_x17_4_254.npy
../apnea/seg\x17\N_x17_4_255.npy
../apnea/seg\x17\N_x17_4_256.npy
../apnea/seg\x17\N_x17_4_257.npy
../apnea/seg\x17\N_x17_4_258.npy
../apnea/seg\x17\N_x17_4_259.npy
../apnea/seg\x17\N_x17_4_260.npy
../apnea/seg\x17\N_x17_4_261.npy
../apnea/seg\x17\N_x17_4_262.npy
../apnea/seg\x17\N_x17_4_263.npy
../apnea/seg\x17\N_x17_4_264.npy
../apnea/seg\x17\N_x17_4_265.npy
../apnea/seg\x17\N_x17_4_266.npy
../apnea/seg\x17\N_x17_4_267.npy
../apnea/seg\x17\N_x17_4_268.npy
../apnea/seg\x17\N_x17_4_269.npy
../apnea/seg\x17\N_x17_4_270.npy
../apnea/seg\x17\N_x17_4_271.npy
../apnea/seg\x17\N_x17_4_272.npy
../apnea/seg\x17\N_x17_4_273.npy
../apnea/s

../apnea/seg\x33\N_x33_4_253.npy
../apnea/seg\x33\N_x33_4_255.npy
../apnea/seg\x33\N_x33_4_256.npy
../apnea/seg\x33\N_x33_4_257.npy
../apnea/seg\x33\N_x33_4_259.npy
../apnea/seg\x33\N_x33_4_260.npy
../apnea/seg\x33\N_x33_4_261.npy
../apnea/seg\x33\N_x33_4_262.npy
../apnea/seg\x33\N_x33_4_263.npy
../apnea/seg\x33\N_x33_4_264.npy
../apnea/seg\x33\N_x33_4_265.npy
../apnea/seg\x33\N_x33_4_266.npy
../apnea/seg\x33\N_x33_4_267.npy
../apnea/seg\x33\N_x33_4_268.npy
../apnea/seg\x33\N_x33_4_269.npy
../apnea/seg\x33\N_x33_4_270.npy
../apnea/seg\x33\N_x33_4_271.npy
../apnea/seg\x33\N_x33_4_272.npy
../apnea/seg\x33\N_x33_4_273.npy
../apnea/seg\x33\N_x33_4_274.npy
../apnea/seg\x33\N_x33_4_275.npy
../apnea/seg\x33\N_x33_4_276.npy
../apnea/seg\x33\N_x33_4_277.npy
../apnea/seg\x33\N_x33_4_278.npy
../apnea/seg\x33\N_x33_4_279.npy
../apnea/seg\x33\N_x33_4_280.npy
../apnea/seg\x33\N_x33_4_281.npy
../apnea/seg\x33\N_x33_4_282.npy
../apnea/seg\x33\N_x33_4_283.npy
../apnea/seg\x33\N_x33_4_284.npy
../apnea/s

../apnea/seg\x34\N_x34_5_324.npy
../apnea/seg\x34\N_x34_5_325.npy
../apnea/seg\x34\N_x34_5_326.npy
../apnea/seg\x34\N_x34_5_327.npy
../apnea/seg\x34\N_x34_5_328.npy
../apnea/seg\x34\N_x34_5_329.npy
../apnea/seg\x34\N_x34_5_330.npy
../apnea/seg\x34\N_x34_5_331.npy
../apnea/seg\x34\N_x34_5_332.npy
../apnea/seg\x34\N_x34_5_333.npy
../apnea/seg\x34\N_x34_5_334.npy
../apnea/seg\x34\N_x34_5_353.npy
../apnea/seg\x34\N_x34_5_354.npy
../apnea/seg\x34\N_x34_5_355.npy
../apnea/seg\x34\N_x34_5_356.npy
../apnea/seg\x34\N_x34_5_357.npy
../apnea/seg\x34\N_x34_5_358.npy
../apnea/seg\x34\N_x34_5_359.npy
../apnea/seg\x34\N_x34_6_360.npy
../apnea/seg\x34\N_x34_6_361.npy
../apnea/seg\x34\N_x34_6_362.npy
../apnea/seg\x34\N_x34_6_363.npy
../apnea/seg\x34\N_x34_6_364.npy
../apnea/seg\x34\N_x34_6_366.npy
../apnea/seg\x34\N_x34_6_367.npy
../apnea/seg\x34\N_x34_6_368.npy
../apnea/seg\x34\N_x34_6_369.npy
../apnea/seg\x34\N_x34_6_370.npy
../apnea/seg\x34\N_x34_6_371.npy
../apnea/seg\x34\N_x34_6_372.npy
../apnea/s

In [42]:
from sklearn.model_selection import KFold
def getTrainTestList(reclist,opt='rec_cv',fold=1):
    train_dtlist=[]
    test_dtlist=[]
    if opt=='rec_cv':#reclist is rec list
        kf = KFold(n_splits=10)
        kf_idx=kf.split(reclist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_rec,test_rec=list(np.array(reclist)[train_idx]),list(np.array(reclist)[test_idx])
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')
    elif opt=='physionet':
        train_rec=[rec for rec in reclist if rec[-3]!='x']
        test_rec=[rec for rec in reclist if rec[-3]=='x']
        print(len(train_rec),len(test_rec))
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')
    elif opt=='physionet_train':
        train_rec=[rec for rec in reclist if rec[-3]!='x']
        #print(len(train_rec),len(test_rec))
        kf_idx=kf.split(dtlist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_dtlist,test_dtlist=list(np.array(dtlist)[train_idx]),list(np.array(dtlist)[test_idx])
    elif opt=='blind':
        subjects=[['a11'],
                  ['a15','x27','x28'],
                  ['a17','x12'],
                  ['b01','x03'],
                  ['c07','x34'],
                  ['a11','a15','x27','x28','a17','x12','b01','x03','c07','x34'],
                  ['a14','a19','x05','x08','x25','b05','x11','c01','x35','c07','x34'],
                  ['a04','a19','x05','x08','x25','b05','x11','c01','x35','c09'],
                  ['b02','b03','x16','x21']]
        train_rec=[rec for rec in reclist if rec.split('/')[-1] not in subjects[fold-1]]
        test_rec=[rec for rec in reclist if rec.split('/')[-1] in subjects[fold-1]]
        print(len(train_rec),len(test_rec),test_rec)
        for rec in train_rec:
            train_dtlist+=glob(rec+'/*.npy*')
        for rec in test_rec:
            test_dtlist+=glob(rec+'/*.npy*')

    else:#reclist is npy data list
        kf = KFold(n_splits=10)
        kf_idx=kf.split(dtlist)
        for fold_idx in range(fold):#1,2,3,4,5
            train_idx,test_idx=kf_idx.__next__()
        train_dtlist,test_dtlist=list(np.array(dtlist)[train_idx]),list(np.array(dtlist)[test_idx])
         
    return train_dtlist,test_dtlist

def dtclean(dt_path):
    dt=np.load(dt_path)
    if dt.std()<0.1:
        print(dt_path)
        return 1      
    else:
        return 0
#train_rec,test_rec=reclist[train_idx],reclist[test_idx]
train_dtlist,test_dtlist=getTrainTestList(reclist,opt='kfold',fold=6)#fold 1-10
print('*********')
print(len(train_dtlist),len(test_dtlist))
train_dtlist=[file for file in train_dtlist if dtclean(file)==0]
np.random.shuffle(train_dtlist)
test_dtlist=[file for file in test_dtlist if dtclean(file)==0]
print(len(train_dtlist),len(test_dtlist))

*********
30882 3431
./seg/a12/A_a12_112.npy
./seg/a12/A_a12_143.npy
./seg/a12/A_a12_144.npy
./seg/a12/A_a12_145.npy
./seg/a12/A_a12_163.npy
./seg/a12/A_a12_176.npy
./seg/a12/A_a12_177.npy
./seg/a12/A_a12_178.npy
./seg/a12/A_a12_179.npy
./seg/a12/A_a12_180.npy
./seg/a12/A_a12_181.npy
./seg/a12/A_a12_182.npy
./seg/a12/A_a12_183.npy
./seg/a12/A_a12_187.npy
./seg/a12/A_a12_191.npy
./seg/a12/A_a12_192.npy
./seg/a12/A_a12_193.npy
./seg/a12/A_a12_194.npy
./seg/a12/A_a12_195.npy
./seg/a12/A_a12_197.npy
./seg/a12/A_a12_206.npy
./seg/a12/A_a12_207.npy
./seg/a12/A_a12_208.npy
./seg/a12/A_a12_209.npy
./seg/a12/A_a12_210.npy
./seg/a12/A_a12_211.npy
./seg/a12/A_a12_212.npy
./seg/a12/A_a12_228.npy
./seg/a12/A_a12_229.npy
./seg/a12/A_a12_230.npy
./seg/a12/A_a12_231.npy
./seg/a12/A_a12_232.npy
./seg/a12/A_a12_233.npy
./seg/a12/A_a12_234.npy
./seg/a12/A_a12_235.npy
./seg/a12/A_a12_237.npy
./seg/a12/A_a12_238.npy
./seg/a12/A_a12_239.npy
./seg/a12/A_a12_240.npy
./seg/a12/A_a12_241.npy
./seg/a12/A_a12_242

./seg/b04/N_b04_5.npy
./seg/b04/N_b04_6.npy
./seg/c01/N_c01_0.npy
./seg/c01/N_c01_1.npy
./seg/c01/N_c01_2.npy
./seg/c01/N_c01_3.npy
./seg/c01/N_c01_4.npy
./seg/c01/N_c01_6.npy
./seg/c01/N_c01_7.npy
./seg/c02/N_c02_0.npy
./seg/c02/N_c02_1.npy
./seg/c02/N_c02_2.npy
./seg/c02/N_c02_3.npy
./seg/c02/N_c02_5.npy
./seg/c02/N_c02_6.npy
./seg/c02/N_c02_7.npy
./seg/c02/N_c02_8.npy
./seg/c04/N_c04_0.npy
./seg/c04/N_c04_1.npy
./seg/c04/N_c04_10.npy
./seg/c04/N_c04_11.npy
./seg/c04/N_c04_12.npy
./seg/c04/N_c04_15.npy
./seg/c04/N_c04_16.npy
./seg/c04/N_c04_17.npy
./seg/c04/N_c04_2.npy
./seg/c04/N_c04_3.npy
./seg/c04/N_c04_4.npy
./seg/c04/N_c04_5.npy
./seg/c04/N_c04_6.npy
./seg/c04/N_c04_7.npy
./seg/c04/N_c04_8.npy
./seg/c04/N_c04_9.npy
./seg/c07/A_c07_227.npy
./seg/c07/A_c07_349.npy
./seg/c07/A_c07_416.npy
./seg/c07/N_c07_149.npy
./seg/c07/N_c07_150.npy
./seg/c07/N_c07_151.npy
./seg/c07/N_c07_152.npy
./seg/c07/N_c07_153.npy
./seg/c07/N_c07_154.npy
./seg/c07/N_c07_155.npy
./seg/c07/N_c07_156.npy
./se

./seg/x17/N_x17_3_222.npy
./seg/x17/N_x17_3_223.npy
./seg/x17/N_x17_3_224.npy
./seg/x17/N_x17_3_225.npy
./seg/x17/N_x17_3_226.npy
./seg/x17/N_x17_3_227.npy
./seg/x17/N_x17_3_228.npy
./seg/x17/N_x17_3_229.npy
./seg/x17/N_x17_3_230.npy
./seg/x17/N_x17_3_231.npy
./seg/x17/N_x17_3_232.npy
./seg/x17/N_x17_3_233.npy
./seg/x17/N_x17_3_234.npy
./seg/x17/N_x17_3_235.npy
./seg/x17/N_x17_3_236.npy
./seg/x17/N_x17_3_237.npy
./seg/x17/N_x17_3_238.npy
./seg/x17/N_x17_3_239.npy
./seg/x17/N_x17_4_240.npy
./seg/x17/N_x17_4_241.npy
./seg/x17/N_x17_4_242.npy
./seg/x17/N_x17_4_243.npy
./seg/x17/N_x17_4_244.npy
./seg/x17/N_x17_4_245.npy
./seg/x17/N_x17_4_246.npy
./seg/x17/N_x17_4_248.npy
./seg/x17/N_x17_4_249.npy
./seg/x17/N_x17_4_250.npy
./seg/x17/N_x17_4_251.npy
./seg/x17/N_x17_4_252.npy
./seg/x17/N_x17_4_253.npy
./seg/x17/N_x17_4_254.npy
./seg/x17/N_x17_4_255.npy
./seg/x17/N_x17_4_256.npy
./seg/x17/N_x17_4_257.npy
./seg/x17/N_x17_4_258.npy
./seg/x17/N_x17_4_259.npy
./seg/x17/N_x17_4_260.npy
./seg/x17/N_

./seg/x34/N_x34_0_3.npy
./seg/x34/N_x34_0_4.npy
./seg/x34/N_x34_0_5.npy
./seg/x34/N_x34_0_7.npy
./seg/x34/N_x34_1_100.npy
./seg/x34/N_x34_1_101.npy
./seg/x34/N_x34_1_102.npy
./seg/x34/N_x34_1_103.npy
./seg/x34/N_x34_1_104.npy
./seg/x34/N_x34_1_105.npy
./seg/x34/N_x34_1_106.npy
./seg/x34/N_x34_1_107.npy
./seg/x34/N_x34_1_108.npy
./seg/x34/N_x34_1_109.npy
./seg/x34/N_x34_1_110.npy
./seg/x34/N_x34_1_111.npy
./seg/x34/N_x34_1_112.npy
./seg/x34/N_x34_1_113.npy
./seg/x34/N_x34_1_114.npy
./seg/x34/N_x34_1_115.npy
./seg/x34/N_x34_1_116.npy
./seg/x34/N_x34_1_117.npy
./seg/x34/N_x34_1_118.npy
./seg/x34/N_x34_1_119.npy
./seg/x34/N_x34_1_90.npy
./seg/x34/N_x34_1_91.npy
./seg/x34/N_x34_1_92.npy
./seg/x34/N_x34_1_93.npy
./seg/x34/N_x34_1_94.npy
./seg/x34/N_x34_1_95.npy
./seg/x34/N_x34_1_96.npy
./seg/x34/N_x34_1_97.npy
./seg/x34/N_x34_1_98.npy
./seg/x34/N_x34_1_99.npy
./seg/x34/N_x34_2_120.npy
./seg/x34/N_x34_2_121.npy
./seg/x34/N_x34_2_122.npy
./seg/x34/N_x34_2_123.npy
./seg/x34/N_x34_2_124.npy
./se

In [8]:
import numpy as np
from torch.utils.data import DataLoader,Dataset
ApneaECGDict={'N':0,
              'A':1}
class ApneaECGDataset(Dataset):
    def __init__(self, filelist, istrain=False):
        # Get the filelist and img data
        self.filelist = filelist
        self.istrain = istrain
        
    def getFeature(self, dt):

        dt=cheb_bandpass_filter(dt, 0.01, 38, 100, 4)
        f, t, Sxx=signal.spectrogram(dt, fs=100.0, window=('hamming'), nperseg=128, noverlap=64, nfft=128, detrend='constant',
        return_onesided=True, scaling='density', axis=-1, mode='psd')
        return Sxx[0:26]
        

    def __getitem__(self, index):
        # return to the data of a Picture
        dt_path = self.filelist[index]
        label = ApneaECGDict.__getitem__(dt_path.split('_')[0][-1])
        data = np.load(dt_path)

        if self.istrain:
            noise = np.random.normal(0, 0.1, data.shape[0])
            data = noise+data
        data=self.getFeature(data)
        return data, label

    def __len__(self):
        return len(self.filelist)


    
class MyLSTM(nn.Module):
    def __init__(self, class_num,fs):
        super(MyLSTM, self).__init__()

#25 * 65
#12* 32
#b*64*6*16
#b*6*64*16
        self.branch1=nn.Sequential(nn.Conv2d(1,64,kernel_size=3,dilation=1,stride=1,padding=1, bias=False),
                                nn.Conv2d(64,64,kernel_size=3,dilation=1,stride=1,padding=1, bias=False),
                     
                                nn.LayerNorm([64,26,92],elementwise_affine=False),
                                nn.ReLU(), 
                                nn.Dropout(p=0.25),
                                nn.MaxPool2d(kernel_size=2,stride=2),

                                nn.LayerNorm([64,13,46],elementwise_affine=False),
                                nn.ReLU(), 
                                nn.Dropout(p=0.5),
                                nn.Conv2d(64,64,kernel_size=3,dilation=1,stride=1,padding=1, bias=False),                                                         
                                nn.MaxPool2d(kernel_size=2,stride=2),

                                nn.LayerNorm([64,6,23],elementwise_affine=False),
                                nn.ReLU(),
                                nn.Dropout(p=0.5),
                                nn.Conv2d(64,64,kernel_size=3,dilation=1,stride=1,padding=1, bias=False))
        
        self.avg = nn.Sequential(nn.AdaptiveMaxPool2d((8,8)),nn.LayerNorm([6,8,8],elementwise_affine=False))
    
        self.lstm = nn.LSTM(64, 8, 2,
                            bias=False,
                            batch_first=True,
                            dropout=0.5,
                            bidirectional=True)
        
        self.fc=nn.Sequential(nn.Linear(16*2,class_num))
    def attention_net(self,lstm_output, final_state):
        hidden = final_state.view(-1, 16, 2)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
        #print('hi',hidden.shape)
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
        #print('att',attn_weights.shape)
        soft_attn_weights = F.softmax(attn_weights, 1)

        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights)
        return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]
    def forward(self, x):
        x=x.unsqueeze(dim=1)
        out=self.branch1(x)
        out=out.permute(0,2,1,3)
        out=self.avg(out)
        #print(out.shape)
        out=out.view(out.shape[0],out.shape[1],-1)
        out,(h,c) = self.lstm(out)
        out,attn=self.attention_net(out, h)
        

        out = self.fc(out.reshape(out.shape[0],-1))
        return out,attn

In [9]:
model=MyLSTM(2,100)

for m in model.modules():
    if isinstance(m, nn.Conv2d):
        if(m.in_channels!=m.out_channels or m.out_channels!=m.groups or m.bias is not None):
            # don't want to reinitialize downsample layers, code assuming normal conv layers will not have these characteristics
            #nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            nn.init.normal_(m.weight, mean=0, std=0.1)
            print(m,'init')
        else:
            print('Not initializing')
criterion=nn.CrossEntropyLoss()
opt=torch.optim.Adam(model.parameters(), lr=0.0003, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)

loss_list=[]

Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) init
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) init
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) init
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) init


In [10]:
def eval(model,test_dtlist,criterion):
    test_dataset = ApneaECGDataset(test_dtlist,istrain=False)
    test_loader=DataLoader(test_dataset, batch_size=256, shuffle=False, sampler=None, num_workers=0)

    sft=nn.Softmax()
    all_pred_prob=torch.Tensor([])
    all_label=torch.Tensor([]).long()
    flag=0
    with torch.no_grad():
        for fe, label in test_loader:
            fe=fe.float()
            #fe=(fe-fe_mean)/fe_std
            label=label.long()
            pred_prob_no_softmax,_=model(fe)
            all_pred_prob=torch.cat((all_pred_prob,pred_prob_no_softmax),0)
            all_label=torch.cat((all_label,label),0)
            
            loss=criterion(pred_prob_no_softmax,label)
            flag+=1
            if flag%8==0:
                print('Eval Loss: ',loss.item())
            #pos=(sft(pred_prob_no_softmax)[:,1]>0.5)
            #print("Acc: ", sum(label==pos).detach().numpy()/len(label))
    all_pred=sft(all_pred_prob)[:,1].detach().numpy()
    all_pred[all_pred>0.5]=1
    all_pred[all_pred<=0.5]=0
    all_label=all_label.detach().numpy()

    print(confusion_matrix(all_label,all_pred))
    print(classification_report(all_label,all_pred))
    print("acc: ",accuracy_score(all_label,all_pred))
    print("pre: ",precision_score(all_label,all_pred))
    print("rec: ",recall_score(all_label,all_pred))
    print("ma F1: ",f1_score(all_label,all_pred, average='macro'))
    print("mi F1: ",f1_score(all_label,all_pred, average='micro'))
    print("we F1: ",f1_score(all_label,all_pred, average='weighted'))
    return accuracy_score(all_label,all_pred)

In [11]:
#Use a good model to demonstrate explainability
ma=0.8
opt.zero_grad()
epoch_flag=0
for epoch in range(26):
    print('epoch: ', epoch)
    if epoch_flag%3==0:# and epoch_flag!=0:
        model.eval()
        atmp=eval(model,test_dtlist,criterion)
        tmp=eval(model,train_dtlist,criterion)
        
        if atmp >ma:
            ma=atmp.copy()
            torch.save(model.state_dict(),'param.pkl')
            print(atmp,ma)
        model.train()
    train_dataset = ApneaECGDataset(train_dtlist,istrain=False)
    train_loader=DataLoader(train_dataset, batch_size=256, shuffle=True, sampler=None, num_workers=0)
    len(train_dataset),len(train_loader)#,len(test_dataset)
    flag=0
    for fe, label in train_loader:
        fe=fe.float()
        
        label=label.long()
        pred_prob_no_softmax,_=model(fe)
        loss=criterion(pred_prob_no_softmax,label)#/256

        loss_list.append(loss.item())
        loss.backward()
        opt.step()
        opt.zero_grad()
        flag+=1
        if flag%4==0:
            print("Loss: ", loss.item())
        
        #print('step')
    print(epoch_flag,'*'*10)
    epoch_flag+=1
    #scheduler.step()

plt.plot(loss_list)
plt.show()
plt.close('all')
print('------')

epoch:  0
Eval Loss:  0.7132495641708374


C:\Users\Talha\miniconda3\envs\srop\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[1805  427]
 [ 993  187]]
              precision    recall  f1-score   support

           0       0.65      0.81      0.72      2232
           1       0.30      0.16      0.21      1180

    accuracy                           0.58      3412
   macro avg       0.47      0.48      0.46      3412
weighted avg       0.53      0.58      0.54      3412

acc:  0.5838218053927315
pre:  0.30456026058631924
rec:  0.15847457627118644
ma F1:  0.46308326185584375
mi F1:  0.5838218053927315
we F1:  0.5415857017821762
Eval Loss:  0.6957210302352905
Eval Loss:  0.6940882802009583
Eval Loss:  0.6918751001358032
Eval Loss:  0.690630316734314
Eval Loss:  0.6927139759063721
Eval Loss:  0.6939866542816162
Eval Loss:  0.6936911344528198
Eval Loss:  0.6956602931022644
Eval Loss:  0.6933732032775879
Eval Loss:  0.6935855150222778
Eval Loss:  0.693444550037384
Eval Loss:  0.6953710317611694
Eval Loss:  0.6963475346565247
Eval Loss:  0.6931898593902588


C:\Users\Talha\miniconda3\envs\srop\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[12895  5037]
 [ 8791  2756]]
              precision    recall  f1-score   support

           0       0.59      0.72      0.65     17932
           1       0.35      0.24      0.29     11547

    accuracy                           0.53     29479
   macro avg       0.47      0.48      0.47     29479
weighted avg       0.50      0.53      0.51     29479

acc:  0.5309203161572645
pre:  0.3536507121775953
rec:  0.23867671256603445
ma F1:  0.4679859514621094
mi F1:  0.5309203161572645
we F1:  0.5076186488265996
Loss:  0.6964522004127502
Loss:  0.6914310455322266
Loss:  0.6908661723136902
Loss:  0.6856143474578857
Loss:  0.6839513778686523
Loss:  0.6859867572784424
Loss:  0.6815207004547119
Loss:  0.6813426613807678
Loss:  0.6735143661499023
Loss:  0.6753376722335815
Loss:  0.6780921220779419
Loss:  0.6739144921302795
Loss:  0.673686683177948
Loss:  0.6604794263839722
Loss:  0.6717765927314758
Loss:  0.6756527423858643
Loss:  0.6588238477706909
Loss:  0.6577273011207581
Loss:  0.641535699

C:\Users\Talha\miniconda3\envs\srop\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[1803  429]
 [ 111 1069]]
              precision    recall  f1-score   support

           0       0.94      0.81      0.87      2232
           1       0.71      0.91      0.80      1180

    accuracy                           0.84      3412
   macro avg       0.83      0.86      0.83      3412
weighted avg       0.86      0.84      0.85      3412

acc:  0.8417350527549824
pre:  0.7136181575433912
rec:  0.9059322033898305
ma F1:  0.8340554812812552
mi F1:  0.8417350527549824
we F1:  0.8450621695515019
Eval Loss:  0.3190448582172394
Eval Loss:  0.44900232553482056
Eval Loss:  0.3984573185443878
Eval Loss:  0.44115957617759705
Eval Loss:  0.42583611607551575
Eval Loss:  0.41223227977752686
Eval Loss:  0.42554089426994324
Eval Loss:  0.4406932294368744
Eval Loss:  0.4476407468318939
Eval Loss:  0.47996848821640015
Eval Loss:  0.45136672258377075
Eval Loss:  0.45498186349868774
Eval Loss:  0.4982185959815979
Eval Loss:  0.4378127157688141


C:\Users\Talha\miniconda3\envs\srop\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[[15337  2595]
 [ 3096  8451]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84     17932
           1       0.77      0.73      0.75     11547

    accuracy                           0.81     29479
   macro avg       0.80      0.79      0.80     29479
weighted avg       0.81      0.81      0.81     29479

acc:  0.8069473184300688
pre:  0.765073329712113
rec:  0.7318784099766174
ma F1:  0.7958055948150915
mi F1:  0.8069473184300688
we F1:  0.8061366876529699
0.8417350527549824 0.8417350527549824
Loss:  0.45145362615585327
Loss:  0.43966972827911377
Loss:  0.458193838596344
Loss:  0.47121065855026245
Loss:  0.5077332258224487
Loss:  0.5010271072387695
Loss:  0.4456416666507721
Loss:  0.47801491618156433
Loss:  0.39348503947257996
Loss:  0.43628960847854614
Loss:  0.41002869606018066
Loss:  0.41541939973831177
Loss:  0.44543394446372986
Loss:  0.43860307335853577
Loss:  0.42074450850486755
Loss:  0.4620485007762909
Loss:  0.39968413114547

KeyboardInterrupt: 

In [12]:
model.load_state_dict(torch.load('param.pkl'))
train_dtlist=[file for file in dtlist if (dtclean(file)==0) and file.split('/')[-1][0]=='A']
train_dataset = ApneaECGDataset(train_dtlist,istrain=False)#In fact it is a test_loader.
train_loader=DataLoader(train_dataset, batch_size=256, shuffle=False, sampler=None, num_workers=0)
len(train_dataset),len(train_loader)#,len(test_dataset)
flag=0
model.eval()
ftmp_=[]
btmp_=[]
with torch.no_grad():
    for fe, label in train_loader:
        fe=fe.float()
        pred_prob_no_softmax,_=model(fe)
        ftmp_.append(_[:,:,0])
        btmp_.append(_[:,:,1])

../apnea/seg\a12\A_a12_112.npy
../apnea/seg\a12\A_a12_143.npy
../apnea/seg\a12\A_a12_144.npy
../apnea/seg\a12\A_a12_145.npy
../apnea/seg\a12\A_a12_163.npy
../apnea/seg\a12\A_a12_176.npy
../apnea/seg\a12\A_a12_177.npy
../apnea/seg\a12\A_a12_178.npy
../apnea/seg\a12\A_a12_179.npy
../apnea/seg\a12\A_a12_180.npy
../apnea/seg\a12\A_a12_181.npy
../apnea/seg\a12\A_a12_182.npy
../apnea/seg\a12\A_a12_183.npy
../apnea/seg\a12\A_a12_187.npy
../apnea/seg\a12\A_a12_191.npy
../apnea/seg\a12\A_a12_192.npy
../apnea/seg\a12\A_a12_193.npy
../apnea/seg\a12\A_a12_194.npy
../apnea/seg\a12\A_a12_195.npy
../apnea/seg\a12\A_a12_197.npy
../apnea/seg\a12\A_a12_206.npy
../apnea/seg\a12\A_a12_207.npy
../apnea/seg\a12\A_a12_208.npy
../apnea/seg\a12\A_a12_209.npy
../apnea/seg\a12\A_a12_210.npy
../apnea/seg\a12\A_a12_211.npy
../apnea/seg\a12\A_a12_212.npy
../apnea/seg\a12\A_a12_228.npy
../apnea/seg\a12\A_a12_229.npy
../apnea/seg\a12\A_a12_230.npy
../apnea/seg\a12\A_a12_231.npy
../apnea/seg\a12\A_a12_232.npy
../apnea

../apnea/seg\a17\A_a17_365.npy
../apnea/seg\b04\N_b04_0.npy
../apnea/seg\b04\N_b04_1.npy
../apnea/seg\b04\N_b04_160.npy
../apnea/seg\b04\N_b04_161.npy
../apnea/seg\b04\N_b04_162.npy
../apnea/seg\b04\N_b04_163.npy
../apnea/seg\b04\N_b04_164.npy
../apnea/seg\b04\N_b04_165.npy
../apnea/seg\b04\N_b04_166.npy
../apnea/seg\b04\N_b04_167.npy
../apnea/seg\b04\N_b04_168.npy
../apnea/seg\b04\N_b04_169.npy
../apnea/seg\b04\N_b04_170.npy
../apnea/seg\b04\N_b04_171.npy
../apnea/seg\b04\N_b04_172.npy
../apnea/seg\b04\N_b04_173.npy
../apnea/seg\b04\N_b04_176.npy
../apnea/seg\b04\N_b04_177.npy
../apnea/seg\b04\N_b04_178.npy
../apnea/seg\b04\N_b04_179.npy
../apnea/seg\b04\N_b04_180.npy
../apnea/seg\b04\N_b04_181.npy
../apnea/seg\b04\N_b04_182.npy
../apnea/seg\b04\N_b04_183.npy
../apnea/seg\b04\N_b04_184.npy
../apnea/seg\b04\N_b04_185.npy
../apnea/seg\b04\N_b04_186.npy
../apnea/seg\b04\N_b04_187.npy
../apnea/seg\b04\N_b04_188.npy
../apnea/seg\b04\N_b04_189.npy
../apnea/seg\b04\N_b04_190.npy
../apnea/seg

../apnea/seg\c10\N_c10_6.npy
../apnea/seg\c10\N_c10_7.npy
../apnea/seg\x01\N_x01_0_0.npy
../apnea/seg\x01\N_x01_0_1.npy
../apnea/seg\x01\N_x01_0_2.npy
../apnea/seg\x01\N_x01_0_3.npy
../apnea/seg\x01\N_x01_0_4.npy
../apnea/seg\x01\N_x01_0_5.npy
../apnea/seg\x01\N_x01_0_6.npy
../apnea/seg\x01\N_x01_0_7.npy
../apnea/seg\x01\N_x01_0_8.npy
../apnea/seg\x01\N_x01_0_9.npy
../apnea/seg\x04\N_x04_0_0.npy
../apnea/seg\x04\N_x04_0_1.npy
../apnea/seg\x04\N_x04_0_2.npy
../apnea/seg\x04\N_x04_0_4.npy
../apnea/seg\x04\N_x04_0_5.npy
../apnea/seg\x04\N_x04_0_6.npy
../apnea/seg\x06\N_x06_0_1.npy
../apnea/seg\x06\N_x06_0_2.npy
../apnea/seg\x06\N_x06_0_3.npy
../apnea/seg\x17\A_x17_6_385.npy
../apnea/seg\x17\N_x17_0_0.npy
../apnea/seg\x17\N_x17_0_1.npy
../apnea/seg\x17\N_x17_0_10.npy
../apnea/seg\x17\N_x17_0_11.npy
../apnea/seg\x17\N_x17_0_12.npy
../apnea/seg\x17\N_x17_0_13.npy
../apnea/seg\x17\N_x17_0_14.npy
../apnea/seg\x17\N_x17_0_15.npy
../apnea/seg\x17\N_x17_0_16.npy
../apnea/seg\x17\N_x17_0_17.npy
..

../apnea/seg\x17\N_x17_5_335.npy
../apnea/seg\x17\N_x17_5_336.npy
../apnea/seg\x17\N_x17_5_337.npy
../apnea/seg\x17\N_x17_5_338.npy
../apnea/seg\x17\N_x17_5_339.npy
../apnea/seg\x17\N_x17_5_340.npy
../apnea/seg\x17\N_x17_5_341.npy
../apnea/seg\x17\N_x17_5_342.npy
../apnea/seg\x17\N_x17_5_343.npy
../apnea/seg\x17\N_x17_5_344.npy
../apnea/seg\x17\N_x17_5_345.npy
../apnea/seg\x17\N_x17_5_346.npy
../apnea/seg\x17\N_x17_5_347.npy
../apnea/seg\x17\N_x17_5_348.npy
../apnea/seg\x17\N_x17_5_349.npy
../apnea/seg\x17\N_x17_5_350.npy
../apnea/seg\x17\N_x17_5_351.npy
../apnea/seg\x17\N_x17_5_352.npy
../apnea/seg\x17\N_x17_5_353.npy
../apnea/seg\x17\N_x17_5_354.npy
../apnea/seg\x17\N_x17_5_355.npy
../apnea/seg\x17\N_x17_5_356.npy
../apnea/seg\x17\N_x17_5_357.npy
../apnea/seg\x17\N_x17_5_358.npy
../apnea/seg\x17\N_x17_5_359.npy
../apnea/seg\x17\N_x17_6_360.npy
../apnea/seg\x17\N_x17_6_361.npy
../apnea/seg\x17\N_x17_6_362.npy
../apnea/seg\x17\N_x17_6_363.npy
../apnea/seg\x17\N_x17_6_364.npy
../apnea/s

../apnea/seg\x33\N_x33_7_468.npy
../apnea/seg\x33\N_x33_7_469.npy
../apnea/seg\x33\N_x33_7_470.npy
../apnea/seg\x33\N_x33_7_471.npy
../apnea/seg\x33\N_x33_7_472.npy
../apnea/seg\x34\A_x34_2_171.npy
../apnea/seg\x34\A_x34_4_259.npy
../apnea/seg\x34\A_x34_6_373.npy
../apnea/seg\x34\N_x34_0_3.npy
../apnea/seg\x34\N_x34_0_4.npy
../apnea/seg\x34\N_x34_0_5.npy
../apnea/seg\x34\N_x34_0_7.npy
../apnea/seg\x34\N_x34_1_100.npy
../apnea/seg\x34\N_x34_1_101.npy
../apnea/seg\x34\N_x34_1_102.npy
../apnea/seg\x34\N_x34_1_103.npy
../apnea/seg\x34\N_x34_1_104.npy
../apnea/seg\x34\N_x34_1_105.npy
../apnea/seg\x34\N_x34_1_106.npy
../apnea/seg\x34\N_x34_1_107.npy
../apnea/seg\x34\N_x34_1_108.npy
../apnea/seg\x34\N_x34_1_109.npy
../apnea/seg\x34\N_x34_1_110.npy
../apnea/seg\x34\N_x34_1_111.npy
../apnea/seg\x34\N_x34_1_112.npy
../apnea/seg\x34\N_x34_1_113.npy
../apnea/seg\x34\N_x34_1_114.npy
../apnea/seg\x34\N_x34_1_115.npy
../apnea/seg\x34\N_x34_1_116.npy
../apnea/seg\x34\N_x34_1_117.npy
../apnea/seg\x34\N

In [13]:
#attention 分布画图
model.load_state_dict(torch.load('param.pkl'))
train_dtlist=[file for file in dtlist if (dtclean(file)==0) and file.split('/')[-1][0]=='N']
train_dataset = ApneaECGDataset(train_dtlist,istrain=False)
#test_dataset = ApneaECGDataset(test_dtlist,istrain=False)
train_loader=DataLoader(train_dataset, batch_size=256, shuffle=False, sampler=None, num_workers=0)
#test_loader=DataLoader(test_dataset, batch_size=256, shuffle=True, sampler=None, num_workers=0)
len(train_dataset),len(train_loader)#,len(test_dataset)
flag=0
model.eval()
fntmp_=[]
bntmp_=[]
with torch.no_grad():
    for fe, label in train_loader:
        fe=fe.float()
        pred_prob_no_softmax,_=model(fe)
        fntmp_.append(_[:,:,0])
        bntmp_.append(_[:,:,1])
an=fntmp_[0]
for tmppp in fntmp_[1::]:
    an=np.vstack((an,tmppp))
bn=bntmp_[0]
for tmppp in bntmp_[1::]:
    bn=np.vstack((bn,tmppp))
an.shape

../apnea/seg\a12\A_a12_112.npy
../apnea/seg\a12\A_a12_143.npy
../apnea/seg\a12\A_a12_144.npy
../apnea/seg\a12\A_a12_145.npy
../apnea/seg\a12\A_a12_163.npy
../apnea/seg\a12\A_a12_176.npy
../apnea/seg\a12\A_a12_177.npy
../apnea/seg\a12\A_a12_178.npy
../apnea/seg\a12\A_a12_179.npy
../apnea/seg\a12\A_a12_180.npy
../apnea/seg\a12\A_a12_181.npy
../apnea/seg\a12\A_a12_182.npy
../apnea/seg\a12\A_a12_183.npy
../apnea/seg\a12\A_a12_187.npy
../apnea/seg\a12\A_a12_191.npy
../apnea/seg\a12\A_a12_192.npy
../apnea/seg\a12\A_a12_193.npy
../apnea/seg\a12\A_a12_194.npy
../apnea/seg\a12\A_a12_195.npy
../apnea/seg\a12\A_a12_197.npy
../apnea/seg\a12\A_a12_206.npy
../apnea/seg\a12\A_a12_207.npy
../apnea/seg\a12\A_a12_208.npy
../apnea/seg\a12\A_a12_209.npy
../apnea/seg\a12\A_a12_210.npy
../apnea/seg\a12\A_a12_211.npy
../apnea/seg\a12\A_a12_212.npy
../apnea/seg\a12\A_a12_228.npy
../apnea/seg\a12\A_a12_229.npy
../apnea/seg\a12\A_a12_230.npy
../apnea/seg\a12\A_a12_231.npy
../apnea/seg\a12\A_a12_232.npy
../apnea

../apnea/seg\a17\A_a17_365.npy
../apnea/seg\b04\N_b04_0.npy
../apnea/seg\b04\N_b04_1.npy
../apnea/seg\b04\N_b04_160.npy
../apnea/seg\b04\N_b04_161.npy
../apnea/seg\b04\N_b04_162.npy
../apnea/seg\b04\N_b04_163.npy
../apnea/seg\b04\N_b04_164.npy
../apnea/seg\b04\N_b04_165.npy
../apnea/seg\b04\N_b04_166.npy
../apnea/seg\b04\N_b04_167.npy
../apnea/seg\b04\N_b04_168.npy
../apnea/seg\b04\N_b04_169.npy
../apnea/seg\b04\N_b04_170.npy
../apnea/seg\b04\N_b04_171.npy
../apnea/seg\b04\N_b04_172.npy
../apnea/seg\b04\N_b04_173.npy
../apnea/seg\b04\N_b04_176.npy
../apnea/seg\b04\N_b04_177.npy
../apnea/seg\b04\N_b04_178.npy
../apnea/seg\b04\N_b04_179.npy
../apnea/seg\b04\N_b04_180.npy
../apnea/seg\b04\N_b04_181.npy
../apnea/seg\b04\N_b04_182.npy
../apnea/seg\b04\N_b04_183.npy
../apnea/seg\b04\N_b04_184.npy
../apnea/seg\b04\N_b04_185.npy
../apnea/seg\b04\N_b04_186.npy
../apnea/seg\b04\N_b04_187.npy
../apnea/seg\b04\N_b04_188.npy
../apnea/seg\b04\N_b04_189.npy
../apnea/seg\b04\N_b04_190.npy
../apnea/seg

../apnea/seg\c10\N_c10_2.npy
../apnea/seg\c10\N_c10_3.npy
../apnea/seg\c10\N_c10_6.npy
../apnea/seg\c10\N_c10_7.npy
../apnea/seg\x01\N_x01_0_0.npy
../apnea/seg\x01\N_x01_0_1.npy
../apnea/seg\x01\N_x01_0_2.npy
../apnea/seg\x01\N_x01_0_3.npy
../apnea/seg\x01\N_x01_0_4.npy
../apnea/seg\x01\N_x01_0_5.npy
../apnea/seg\x01\N_x01_0_6.npy
../apnea/seg\x01\N_x01_0_7.npy
../apnea/seg\x01\N_x01_0_8.npy
../apnea/seg\x01\N_x01_0_9.npy
../apnea/seg\x04\N_x04_0_0.npy
../apnea/seg\x04\N_x04_0_1.npy
../apnea/seg\x04\N_x04_0_2.npy
../apnea/seg\x04\N_x04_0_4.npy
../apnea/seg\x04\N_x04_0_5.npy
../apnea/seg\x04\N_x04_0_6.npy
../apnea/seg\x06\N_x06_0_1.npy
../apnea/seg\x06\N_x06_0_2.npy
../apnea/seg\x06\N_x06_0_3.npy
../apnea/seg\x17\A_x17_6_385.npy
../apnea/seg\x17\N_x17_0_0.npy
../apnea/seg\x17\N_x17_0_1.npy
../apnea/seg\x17\N_x17_0_10.npy
../apnea/seg\x17\N_x17_0_11.npy
../apnea/seg\x17\N_x17_0_12.npy
../apnea/seg\x17\N_x17_0_13.npy
../apnea/seg\x17\N_x17_0_14.npy
../apnea/seg\x17\N_x17_0_15.npy
../apnea

../apnea/seg\x17\N_x17_5_321.npy
../apnea/seg\x17\N_x17_5_322.npy
../apnea/seg\x17\N_x17_5_323.npy
../apnea/seg\x17\N_x17_5_324.npy
../apnea/seg\x17\N_x17_5_325.npy
../apnea/seg\x17\N_x17_5_326.npy
../apnea/seg\x17\N_x17_5_327.npy
../apnea/seg\x17\N_x17_5_328.npy
../apnea/seg\x17\N_x17_5_329.npy
../apnea/seg\x17\N_x17_5_330.npy
../apnea/seg\x17\N_x17_5_331.npy
../apnea/seg\x17\N_x17_5_332.npy
../apnea/seg\x17\N_x17_5_333.npy
../apnea/seg\x17\N_x17_5_334.npy
../apnea/seg\x17\N_x17_5_335.npy
../apnea/seg\x17\N_x17_5_336.npy
../apnea/seg\x17\N_x17_5_337.npy
../apnea/seg\x17\N_x17_5_338.npy
../apnea/seg\x17\N_x17_5_339.npy
../apnea/seg\x17\N_x17_5_340.npy
../apnea/seg\x17\N_x17_5_341.npy
../apnea/seg\x17\N_x17_5_342.npy
../apnea/seg\x17\N_x17_5_343.npy
../apnea/seg\x17\N_x17_5_344.npy
../apnea/seg\x17\N_x17_5_345.npy
../apnea/seg\x17\N_x17_5_346.npy
../apnea/seg\x17\N_x17_5_347.npy
../apnea/seg\x17\N_x17_5_348.npy
../apnea/seg\x17\N_x17_5_349.npy
../apnea/seg\x17\N_x17_5_350.npy
../apnea/s

../apnea/seg\x33\N_x33_7_421.npy
../apnea/seg\x33\N_x33_7_422.npy
../apnea/seg\x33\N_x33_7_468.npy
../apnea/seg\x33\N_x33_7_469.npy
../apnea/seg\x33\N_x33_7_470.npy
../apnea/seg\x33\N_x33_7_471.npy
../apnea/seg\x33\N_x33_7_472.npy
../apnea/seg\x34\A_x34_2_171.npy
../apnea/seg\x34\A_x34_4_259.npy
../apnea/seg\x34\A_x34_6_373.npy
../apnea/seg\x34\N_x34_0_3.npy
../apnea/seg\x34\N_x34_0_4.npy
../apnea/seg\x34\N_x34_0_5.npy
../apnea/seg\x34\N_x34_0_7.npy
../apnea/seg\x34\N_x34_1_100.npy
../apnea/seg\x34\N_x34_1_101.npy
../apnea/seg\x34\N_x34_1_102.npy
../apnea/seg\x34\N_x34_1_103.npy
../apnea/seg\x34\N_x34_1_104.npy
../apnea/seg\x34\N_x34_1_105.npy
../apnea/seg\x34\N_x34_1_106.npy
../apnea/seg\x34\N_x34_1_107.npy
../apnea/seg\x34\N_x34_1_108.npy
../apnea/seg\x34\N_x34_1_109.npy
../apnea/seg\x34\N_x34_1_110.npy
../apnea/seg\x34\N_x34_1_111.npy
../apnea/seg\x34\N_x34_1_112.npy
../apnea/seg\x34\N_x34_1_113.npy
../apnea/seg\x34\N_x34_1_114.npy
../apnea/seg\x34\N_x34_1_115.npy
../apnea/seg\x34\N

IndexError: list index out of range

In [ ]:
#a=np.array(_[:,:,0].shape)
a=ftmp_[0]
for tmppp in ftmp_[1::]:
    a=np.vstack((a,tmppp))
b=btmp_[0]
for tmppp in btmp_[1::]:
    b=np.vstack((b,tmppp))
a.shape

In [ ]:
#Apnea->all
#Figure 4
sns.set(style='darkgrid',font='Times New Roman')
cmap_=['Greens','hot','spring','summer','autumn','winter','ocean']
f=plt.figure(dpi=256)
f.subplots_adjust(hspace=0.09,wspace=0.05)
for i in range(6):
    plt.subplot(3,4,i+1)
    sns.kdeplot(a[:,i],b[:,i],shade=True,cmap='Greens',shade_lowest=False)
    plt.xlim(0.1,0.25)
    plt.ylim(0.1,0.25)
    plt.legend([i])
    if i in [0,1,2,3,4,5]:
        plt.xticks([])
    if i in [1,5,2,3,0,4]:
        plt.yticks([])
    if i==4:
        plt.ylabel('Backward weights')

for i in range(6):
    plt.subplot(3,4,i+1+6)
    sns.kdeplot(an[:,i],bn[:,i],shade=True,cmap='Blues',shade_lowest=False)
    plt.xlim(0.1,0.25)
    plt.ylim(0.1,0.25)
    plt.legend([i])
    if i in [0,1,3,2,4,5]:
        plt.xticks([])
    if i in [1,3,0,2,4,5]:
        plt.yticks([])

plt.savefig('Atten.pdf',dpi=256)
plt.show()
plt.close("all")

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import numpy as np
bw2=3
filetmp=dtlist[1925]#5
dt=np.load(filetmp)
f=plt.figure(dpi=512)
f.subplots_adjust(hspace=0.2)
ax=plt.subplot(3,2,1)
plt.plot(dt)
#plt.xlim(0,6000)
plt.ylim(-1,4)
plt.xticks([])
ax=plt.gca()
ax.yaxis.set_ticks_position('right')
plt.xlabel('0-60 seconds',rotation=0,ha='center',va='top',font='Times New Roman')

ax=plt.subplot(3,3,3)
plt.ylim(-1,4)
plt.plot(dt[0:500])
plt.xticks([])
plt.ylabel('Voltage ($mv$)',rotation=0,ha='right',va='top',font='Times New Roman')
plt.xlabel('0-5 seconds',rotation=0,ha='center',va='top',font='Times New Roman')

#plt.show()
#plt.close("all")

dt=cheb_bandpass_filter(dt, 0.01, 38, 100, 4)
ax=plt.subplot(3,2,3)
plt.plot(dt)
#plt.xlim(0,6000)
plt.ylim(-1,4)
plt.xticks([])
ax=plt.gca()
ax.yaxis.set_ticks_position('right')
plt.xlabel('0-60 seconds',rotation=0,ha='center',va='top',font='Times New Roman')

ax=plt.subplot(3,3,6)
plt.ylim(-1,4)
plt.plot(dt[0:500])
plt.xticks([])
plt.ylabel('Voltage ($mv$)',rotation=0,ha='right',va='top',font='Times New Roman')
plt.xlabel('0-5 seconds',rotation=0,ha='center',va='top',font='Times New Roman')


f, t, Sxx=signal.spectrogram(dt, fs=100.0, window=('hamming'), nperseg=128, noverlap=64, nfft=128, detrend='constant',
return_onesided=True, scaling='density', axis=-1, mode='psd')
input=torch.FloatTensor(Sxx[0:26]).unsqueeze(dim=0)
# with torch.no_grad():
#     pred_prob_no_softmax,_=model(torch.FloatTensor(input))


ax=plt.subplot(3,1,3)
plt.imshow(Sxx,aspect='auto',interpolation='nearest')
plt.plot([26]*92,'r--',linewidth=3)
ax=plt.gca()
ax.yaxis.set_ticks_position('left')
plt.xlim(0,91)
plt.ylim(0,50)
plt.xticks([])
plt.yticks([])
plt.xlabel('0-60 seconds',rotation=0,ha='center',va='top',font='Times New Roman')
#plt.ylabel('Frequency ($Hz$)',rotation=0,ha='right')
cb=plt.colorbar(orientation='vertical',fraction=0.2)

plt.tight_layout()
plt.savefig('NewApneaScheme.pdf',dpi=512)
plt.show()
plt.close("all")

print(filetmp)

In [ ]:
#统计十折交叉验证结果并画图
#画完图用ppt补字
acc=[0.8561004530114621,0.8548627367525006,0.8625763886777538,
     0.8490476852495482,0.8640661579155392,0.8639141406463774,
     0.8571645738955945,0.8597488674713447,0.8615520282186949]#28 38 48/32 64 128
fone=[0.8004385040266475,0.8009506337558372,0.8167815160113499,
      0.7915210629392112,0.8161217355541847,0.8175890455619855,
      0.8004926108374384,0.8084540962504672,0.806264831514001]#28 38 48/32 64 128
sen=[0.7458159817710379,0.7546947434587884,0.7916241062308478,
      0.7507284382284383,0.7796024200518582,0.7986464968152867,
      0.7405515832482125,0.7649092480553155,0.7679233342374108]#28 38 48/32 64 128

spe=[0.9257091846855783,0.9180799365268273,0.907359650862924,
       0.9097450195619913,0.9173775044634002,0.9042349122030396,
      0.9307677048204721,0.9196092045229122,0.9177658361830321]

In [ ]:
# TN=1667+1646+1903+1997+2022+3064+3069+1234+1134
# FN=84+96+170+130+210+33+68+302+296
# FP=161+472+321+608+168+36+212+550+498
# TP=1108+1194+680+696+1012+85+22+1345+1166# [[1134  296]
#  [ 498 1166]]

In [ ]:
#10 fold 

#fold=10
# epoch:  27
# Eval Loss:  0.3914395570755005
# [[1667   84]
#  [ 161 1108]]
#               precision    recall  f1-score   support

#            0       0.91      0.95      0.93      1751
#            1       0.93      0.87      0.90      1269

#     accuracy                           0.92      3020
#    macro avg       0.92      0.91      0.92      3020
# weighted avg       0.92      0.92      0.92      3020

# acc:  0.9188741721854304
# pre:  0.9295302013422819
# rec:  0.8731284475965327
# ma F1:  0.9159960485558507
# mi F1:  0.9188741721854304
# we F1:  0.9184777222400321

#fold=9
# 11 **********
# epoch:  12
# Eval Loss:  1.2395700216293335
# [[1646   96]
#  [ 472 1194]]
#               precision    recall  f1-score   support

#            0       0.78      0.94      0.85      1742
#            1       0.93      0.72      0.81      1666

#     accuracy                           0.83      3408
#    macro avg       0.85      0.83      0.83      3408
# weighted avg       0.85      0.83      0.83      3408

# acc:  0.8333333333333334
# pre:  0.9255813953488372
# rec:  0.7166866746698679
# ma F1:  0.8303490923878367
# mi F1:  0.8333333333333334
# we F1:  0.8308508674140707

#fold=8
# epoch:  27
# Eval Loss:  0.8047547936439514
# [[1903  170]
#  [ 321  680]]
#               precision    recall  f1-score   support

#            0       0.86      0.92      0.89      2073
#            1       0.80      0.68      0.73      1001

#     accuracy                           0.84      3074
#    macro avg       0.83      0.80      0.81      3074
# weighted avg       0.84      0.84      0.84      3074

# acc:  0.8402732595966168
# pre:  0.8
# rec:  0.6793206793206793
# ma F1:  0.81023610632825
# mi F1:  0.8402732595966169
# we F1:  0.8365646658570227
#fold=7
#model.eval()
#eval(model,test_dtlist,criterion)
#another 30 epoch 
#epoch:  3
# Eval Loss:  0.3194165527820587
# [[1997  130]
#  [ 608  696]]
#               precision    recall  f1-score   support

#            0       0.77      0.94      0.84      2127
#            1       0.84      0.53      0.65      1304

#     accuracy                           0.78      3431
#    macro avg       0.80      0.74      0.75      3431
# weighted avg       0.80      0.78      0.77      3431

# acc:  0.784902360827747
# pre:  0.8426150121065376
# rec:  0.5337423312883436
# ma F1:  0.7487808507851845
# mi F1:  0.784902360827747
# we F1:  0.7716309682064212

#fold=6

# epoch:  9
# Eval Loss:  0.2403402030467987
# [[2022  210]
#  [ 168 1012]]
#               precision    recall  f1-score   support

#            0       0.92      0.91      0.91      2232
#            1       0.83      0.86      0.84      1180

#     accuracy                           0.89      3412
#    macro avg       0.88      0.88      0.88      3412
# weighted avg       0.89      0.89      0.89      3412

# acc:  0.8892145369284877
# pre:  0.8281505728314239
# rec:  0.8576271186440678
# ma F1:  0.878574729109731
# mi F1:  0.8892145369284877
# we F1:  0.889656984382337

#fold=5
# epoch:  21
# Eval Loss:  0.08229190856218338
# [[3064   33]
#  [  36   85]]
#               precision    recall  f1-score   support

#            0       0.99      0.99      0.99      3097
#            1       0.72      0.70      0.71       121

#     accuracy                           0.98      3218
#    macro avg       0.85      0.85      0.85      3218
# weighted avg       0.98      0.98      0.98      3218

# acc:  0.9785581106277191
# pre:  0.7203389830508474
# rec:  0.7024793388429752
# ma F1:  0.8500813256245598
# mi F1:  0.9785581106277191
# we F1:  0.9784287281654802

#fold=4
# 20 **********
# epoch:  21
# Eval Loss:  0.1669204831123352
# [[3069   68]
#  [ 212   22]]
#               precision    recall  f1-score   support

#            0       0.94      0.98      0.96      3137
#            1       0.24      0.09      0.14       234

#     accuracy                           0.92      3371
#    macro avg       0.59      0.54      0.55      3371
# weighted avg       0.89      0.92      0.90      3371

# acc:  0.9169385938890537
# pre:  0.24444444444444444
# rec:  0.09401709401709402
# ma F1:  0.5460875854560284
# mi F1:  0.9169385938890537
# we F1:  0.8994123237169647

#fold=3
# 30 epoch
# [[1234  302]
#  [ 550 1345]]
#               precision    recall  f1-score   support

#            0       0.69      0.80      0.74      1536
#            1       0.82      0.71      0.76      1895

#     accuracy                           0.75      3431
#    macro avg       0.75      0.76      0.75      3431
# weighted avg       0.76      0.75      0.75      3431

# acc:  0.7516758962401632
# pre:  0.8166363084395871
# rec:  0.7097625329815304
# ma F1:  0.7514157136734403
# mi F1:  0.7516758962401633
# we F1:  0.7522572050378866

#fold=2
#30epoch
# Eval Loss:  0.3450790345668793
# [[1134  296]
#  [ 498 1166]]
#               precision    recall  f1-score   support

#            0       0.69      0.79      0.74      1430
#            1       0.80      0.70      0.75      1664

#     accuracy                           0.74      3094
#    macro avg       0.75      0.75      0.74      3094
# weighted avg       0.75      0.74      0.74      3094

# acc:  0.7433742727860375
# pre:  0.7975376196990424
# rec:  0.7007211538461539
# ma F1:  0.7433468187632601
# mi F1:  0.7433742727860376
# we F1:  0.7435475763048185
#fold=1
# epoch:  27
# Eval Loss:  0.3398006856441498
# [[ 808  231]
#  [ 470 1923]]
#               precision    recall  f1-score   support

#            0       0.63      0.78      0.70      1039
#            1       0.89      0.80      0.85      2393

#     accuracy                           0.80      3432
#    macro avg       0.76      0.79      0.77      3432
# weighted avg       0.81      0.80      0.80      3432

# acc:  0.7957459207459208
# pre:  0.8927576601671309
# rec:  0.8035938152946093
# ma F1:  0.7716430103881211
# mi F1:  0.7957459207459208
# we F1:  0.8009123741768751

In [ ]:
#fold=1,sub 10
#epoch:  18
# [[205  39]
#  [ 97 125]]
#               precision    recall  f1-score   support

#            0       0.68      0.84      0.75       244
#            1       0.76      0.56      0.65       222

#     accuracy                           0.71       466
#    macro avg       0.72      0.70      0.70       466
# weighted avg       0.72      0.71      0.70       466

# acc:  0.7081545064377682
# pre:  0.7621951219512195
# rec:  0.5630630630630631
# ma F1:  0.6992920723490672
# mi F1:  0.7081545064377682
# we F1:  0.7017292417234599
#----------------------
#fold=2,sub11
# epoch:  3
# [[ 123   92]
#  [ 131 1148]]
#               precision    recall  f1-score   support

#            0       0.48      0.57      0.52       215
#            1       0.93      0.90      0.91      1279

#     accuracy                           0.85      1494
#    macro avg       0.71      0.73      0.72      1494
# weighted avg       0.86      0.85      0.86      1494

# acc:  0.8507362784471219
# pre:  0.9258064516129032
# rec:  0.8975762314308053
# ma F1:  0.7179965312664263
# mi F1:  0.8507362784471219
# we F1:  0.8557868639496069
#--------------
#fold=3,sub12
# epoch:  12
# [[754  43]
#  [ 76 138]]
#               precision    recall  f1-score   support

#            0       0.91      0.95      0.93       797
#            1       0.76      0.64      0.70       214

#     accuracy                           0.88      1011
#    macro avg       0.84      0.80      0.81      1011
# weighted avg       0.88      0.88      0.88      1011

# acc:  0.8822947576656776
# pre:  0.7624309392265194
# rec:  0.6448598130841121
# ma F1:  0.8127967136844235
# mi F1:  0.8822947576656776
# we F1:  0.8785716481666819
#--------
#fold=4 sub =14
# epoch:  6
# [[880  41]
#  [ 25   6]]
#               precision    recall  f1-score   support

#            0       0.97      0.96      0.96       921
#            1       0.13      0.19      0.15        31

#     accuracy                           0.93       952
#    macro avg       0.55      0.57      0.56       952
# weighted avg       0.94      0.93      0.94       952

# acc:  0.930672268907563
# pre:  0.1276595744680851
# rec:  0.1935483870967742
# ma F1:  0.5588507877664504
# mi F1:  0.930672268907563
# we F1:  0.9374790694776519
#--------
#fold=5 sub =24
# epoch:  9
# [[570  16]
#  [  1   1]]
#               precision    recall  f1-score   support

#            0       1.00      0.97      0.99       586
#            1       0.06      0.50      0.11         2

#     accuracy                           0.97       588
#    macro avg       0.53      0.74      0.55       588
# weighted avg       1.00      0.97      0.98       588

# acc:  0.9710884353741497
# pre:  0.058823529411764705
# rec:  0.5
# ma F1:  0.545284992949097
# mi F1:  0.9710884353741497
# we F1:  0.9823134821867611
#-----
#fold=6
# 60 10 ['./seg/a11', './seg/a15', './seg/a17', './seg/b01', './seg/c07', './seg/x03', './seg/x12', './seg/x27', './seg/x28', './seg/x34']
# epoch:  12
# Eval Loss:  0.18583853542804718
# Eval Loss:  0.4881991147994995
# [[2549  214]
#  [ 400 1348]]
#               precision    recall  f1-score   support

#            0       0.86      0.92      0.89      2763
#            1       0.86      0.77      0.81      1748

#     accuracy                           0.86      4511
#    macro avg       0.86      0.85      0.85      4511
# weighted avg       0.86      0.86      0.86      4511

# acc:  0.8638882731101751
# pre:  0.8629961587708067
# rec:  0.7711670480549199
# ma F1:  0.8535042566875692
# mi F1:  0.8638882731101751
# we F1:  0.8622800907166659
#--------
#fold=7
# 59 11 ['./seg/a14', './seg/a19', './seg/b05', './seg/c01', './seg/c07', './seg/x05', './seg/x08', './seg/x11', './seg/x25', './seg/x34', './seg/x35']

# epoch:  9
# Eval Loss:  0.13865183293819427
# Eval Loss:  0.46740415692329407
# [[3023  359]
#  [ 364 1227]]
#               precision    recall  f1-score   support

#            0       0.89      0.89      0.89      3382
#            1       0.77      0.77      0.77      1591

#     accuracy                           0.85      4973
#    macro avg       0.83      0.83      0.83      4973
# weighted avg       0.85      0.85      0.85      4973

# acc:  0.8546149205710839
# pre:  0.7736443883984867
# rec:  0.771213073538655
# ma F1:  0.8328081791525578
# mi F1:  0.8546149205710839
# we F1:  0.8545542113800523
#---
#fold=8
# 60 10 ['./seg/a04', './seg/a19', './seg/b05', './seg/c01',
#        './seg/c09', './seg/x05', './seg/x08', './seg/x11', './seg/x25', './seg/x35']
# 8 **********
# epoch:  9
# Eval Loss:  0.06629161536693573
# Eval Loss:  0.5968064665794373
# [[2858  306]
#  [ 371 1290]]
#               precision    recall  f1-score   support

#            0       0.89      0.90      0.89      3164
#            1       0.81      0.78      0.79      1661

#     accuracy                           0.86      4825
#    macro avg       0.85      0.84      0.84      4825
# weighted avg       0.86      0.86      0.86      4825

# acc:  0.8596891191709845
# pre:  0.8082706766917294
# rec:  0.7766405779650812
# ma F1:  0.8431214656074602
# mi F1:  0.8596891191709845
# we F1:  0.8590023223077006
# #--------
# #fold=9

# # 66 4 ['./seg/b02', './seg/b03', './seg/x16', './seg/x21']
# 14 **********
# epoch:  15
# Eval Loss:  0.34839051961898804
# [[1570   62]
#  [ 224  127]]
#               precision    recall  f1-score   support

#            0       0.88      0.96      0.92      1632
#            1       0.67      0.36      0.47       351

#     accuracy                           0.86      1983
#    macro avg       0.77      0.66      0.69      1983
# weighted avg       0.84      0.86      0.84      1983

# acc:  0.8557740796772567
# pre:  0.671957671957672
# rec:  0.36182336182336183
# ma F1:  0.6934455471233054
# mi F1:  0.8557740796772567
# we F1:  0.8375500864175616

In [25]:
#Value of model parameters and MACS of two models
import torch
from torch import nn

class ResBlock1(nn.Module):

    def __init__(self, in_planes, out_planes):
        super(ResBlock1, self).__init__()
        self.conv1 = nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=1,padding=0)
        self.bn = nn.BatchNorm1d(out_planes)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_planes, out_planes, kernel_size=3, stride=1,padding=1)

        self.downsample = nn.Sequential(nn.AvgPool1d(kernel_size =2, stride = 1,padding=0),
                                        nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=1,padding=0))

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.conv2(out)
        
        out=torch.nn.functional.pad(out,(0,1))
        shortcut = self.downsample(x)

        return out + shortcut

class ResBlock2(nn.Module):

    def __init__(self, in_planes, out_planes):
        super(ResBlock2, self).__init__()
        
        self.bn1 = nn.BatchNorm1d(in_planes)
        self.relu1=nn.ReLU()
        self.dropout1 = nn.Dropout()
        self.conv1 = nn.Conv1d(in_planes, out_planes, kernel_size=3, stride= 1, padding=1)
        
        self.bn2 = nn.BatchNorm1d(out_planes)
        self.relu2=nn.ReLU()
        self.dropout2 = nn.Dropout()
        self.conv2 = nn.Conv1d(out_planes, out_planes, kernel_size=3, stride=1, padding=1)
        self.downsample = nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=1,padding=0)      

    def forward(self, x):
        #print(x.size())
        out = self.bn1(x)
        out = self.relu1(out)
        #out = self.dropout1(out)
        out = self.conv1(out)
        
        out = self.bn2(out)
        out = self.relu2(out)
        #out = self.dropout2(out)
        out = self.conv2(out)
        

        shortcut = self.downsample(x)
        #print(shortcut.shape)
        return out + shortcut


class RRNet(nn.Module):

    def __init__(self, num_classes):
        super(RRNet, self).__init__()
        self.Begin = [nn.Conv1d(1, 128, kernel_size=20, stride= 5, padding=0),
        nn.BatchNorm1d(128),
        nn.ReLU(),nn.AvgPool1d(kernel_size = 2, stride = 1)] 
        
        self.Res1 = [ResBlock1(128,64)]
        self.Res64 = [ResBlock2(64,64),ResBlock2(64,64)]
        self.Res128 = [ResBlock2(64,128),ResBlock2(128,128),ResBlock2(128,128),
                       ResBlock2(128,128)]
        self.Res256 = [ResBlock2(128,256),ResBlock2(256,256),ResBlock2(256,256),
                       ResBlock2(256,256),ResBlock2(256,256),ResBlock2(256,256)]
        self.Res512 = [ResBlock2(256,512),ResBlock2(512,512),ResBlock2(512,512)]
        #self.conv512 = nn.Conv1d(512, 512, kernel_size=3, stride=1, padding=0)
        
        self.End = [nn.AvgPool1d(kernel_size=67,stride=1,padding=0)]
        self.Features = nn.Sequential(*self.Begin,*self.Res1,
                                      *self.Res64,*self.Res128,
                                      *self.Res256,*self.Res512,*self.End)
        self.Out = nn.Sequential(nn.Linear(512, num_classes))

    def forward(self, x):       
        Features = self.Features(x)
        out = self.Out(Features.view(Features.size(0),1,-1))
        out = out.view(out.size(0),-1)
        return out
model=RRNet(2)
inp=torch.FloatTensor(1,1,360)
from thop import profile
from thop import clever_format
with torch.no_grad():
    macs, params = profile(model, inputs=(inp, ))
print(macs, params)
macs, params = clever_format([macs, params], "%.3f")
print(macs, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool1d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.ResBlock1'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class '__main__.ResBlock2'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.RRNet'>. Treat it as zero Macs and zero Params.
546634688.0 8159938.0
546.635M 8.160M
